In [1]:
import pandas as pd
import requests
import datetime
from bs4 import BeautifulSoup
import os
from tqdm import tqdm
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.cluster import KMeans
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


import pandas as pd
from political_utils import clustering as cl
import warnings
warnings.filterwarnings("ignore")

In [2]:
# load a data frame with required columns
df = pd.read_pickle('resources/data/kav_clean_072119.pkl')

In [3]:
df = df[df['article']!='']  # dirty fix

In [4]:
# add number of words column
df['word_count'] = df['article'].apply(lambda x: len(x.split(' ')))

In [5]:
len(df)

4306

In [12]:
# construct cues
hf = pd.read_excel('resources/data/blasey ford politicians names.xlsx')
mention = {}
for c in hf.columns:
    x = set(hf[c])
    for i in list(x):
        if str(i).lower()=='nan':
            x.remove(i)
    mention[c] = [j.lower() for j in list(x)]
    


In [13]:
mention

{'Senators': ['dick durbin',
  'lisa murkowski',
  'mike lee',
  'kamala harris',
  'bob corker',
  'susan collins',
  'dianne feinstein',
  'heidi heitkamp',
  'john kennedy',
  'lindsey graham',
  'chuck schumer',
  'john cornyn',
  'cory booker',
  'patrick leahy',
  'joe manchin',
  'thom tillis',
  'jeff flake',
  'amy klobuchar',
  'ben sasse',
  'kirsten gillibrand',
  'chuck grassley',
  'chris coons',
  'ted cruz',
  'mazie hirono',
  'joe donnelly',
  'mike crapo',
  'charles grassley',
  'bernie sanders',
  'orrin hatch',
  'richard blumenthal',
  'mitch mcconnell',
  'claire mccaskill',
  'sheldon whitehouse',
  'dean heller'],
 'Other politicians': ['kennedy',
  'clarence thomas',
  'george bush',
  'barack obama',
  'neil gorsuch',
  'anna eshoo',
  'phil bredesen',
  'stephen miller',
  'robert bork',
  'rod rosenstein',
  'rachel mitchell',
  'hillary clinton',
  'merrick garland',
  'donald trump',
  'anthony kennedy',
  'kellyanne conway',
  'bill clinton',
  'keith e

In [48]:
def political_cue_stats(cues, article):
    text = str(article).lower()
    stats = {}
    total = 0
    for k, v in cues.items():
        for c in v:
            count = text.count(c)
            if count > 0:
                stats[c] = count
                total += count
    stats['total'] = total
    return stats

In [49]:
df['cue_stats'] = df['article'].apply(lambda x: political_cue_stats(mention, x))
df['cue_stats_title'] = df['title'].apply(lambda x: political_cue_stats(mention, x))

In [42]:
df['cue_stats'][0]

{'dianne feinstein': 1, 'senate': 1, 'house': 2, 'total': 4}

In [50]:
df['total'] = df['cue_stats'].apply(lambda x: x['total'])

In [53]:
df.to_pickle('cue_kav_100919.pkl')

In [52]:
df

,publish_date,url,title,authors,media_site,article,issue,to_drop,word_count,cue_stats,total,cue_stats_title
0,9/20/18,https://www.americanthinker.com/articles/2018/...,Blasey Ford Must Be Acknowledged and Then Dism...,[],americanthinker.com,She released only selected portions of her the...,False,False,1266,"{'dianne feinstein': 1, 'senate': 1, 'house': ...",4,{'total': 0}
1,9/21/18,https://www.americanthinker.com/articles/2018/...,Dear Juanita Broaddrick,[],americanthinker.com,They want an FBI investigation of an individua...,False,False,969,"{'dianne feinstein': 2, 'bill clinton': 4, 'se...",10,{'total': 0}
2,9/25/18,https://www.americanthinker.com/articles/2018/...,Debra Katz Was Wrong about Paula Jones's Case,[],americanthinker.com,Debra Katz Was Wrong about Paula Jones's Case\...,False,False,1444,"{'hillary clinton': 1, 'bill clinton': 1, 'tot...",2,{'total': 0}
3,9/28/18,https://www.americanthinker.com/articles/2018/...,Fake Rape Victims Are More Fun than Real Ones,[],americanthinker.com,Fake Rape Victims Are More Fun than Real Ones\...,False,False,1151,{'total': 0},0,{'total': 0}
4,9/20/18,https://www.americanthinker.com/articles/2018/...,Ford vs. Kavanaugh: There's Nothing to Investi...,[],americanthinker.com,The problem with that is that there is nothing...,False,False,934,"{'kennedy': 1, 'house': 4, 'total': 5}",5,{'total': 0}
5,9/19/18,https://www.americanthinker.com/articles/2018/...,Ford's Accusation against Kavanaugh Is Not Cre...,[],americanthinker.com,Now the Dems and the media say any claim of se...,False,False,1234,"{'dianne feinstein': 2, 'kennedy': 1, 'bill cl...",8,{'total': 0}
6,9/21/18,https://www.americanthinker.com/articles/2018/...,J'accuse as the New Legal Standard,[],americanthinker.com,""" J'accuse "" was the title of an editorial pub...",False,False,1194,"{'dianne feinstein': 1, 'clarence thomas': 1, ...",11,{'total': 0}
7,9/25/18,https://www.americanthinker.com/articles/2018/...,Judge Kavanaugh and Sexual McCarthyism,[],americanthinker.com,McCarthy was looking for communists in the Sta...,False,False,1310,"{'dianne feinstein': 1, 'kennedy': 1, 'donald ...",6,{'total': 0}
8,9/18/18,https://www.americanthinker.com/articles/2018/...,Judge Kavanaugh Meets His Anita Hill,[],americanthinker.com,After the Democrats prattled on about not havi...,False,False,1206,"{'dianne feinstein': 1, 'chuck grassley': 1, '...",13,{'total': 0}
9,9/29/18,https://www.americanthinker.com/articles/2018/...,Kavanaugh Character Assassins Must Pay in Nove...,[],americanthinker.com,From Dianne Feinstein's withholding of Dr. For...,False,False,1326,"{'kamala harris': 1, 'bob corker': 1, 'dianne ...",22,{'total': 0}


In [20]:
df['article'].iloc[0].count('Dianne Feinstein')

1

In [33]:
df['cue_stats'] = df['article'].apply(lambda x: political_cue_stats(mention, x))

In [34]:
df['total'] = df['cue_stats'].apply(lambda x: x['total'])

In [36]:
cue_df = df[df['total']>0]

In [37]:
cue_df

,publish_date,url,title,authors,media_site,article,issue,to_drop,word_count,cue_stats,total
133,9/17/18,http://feedproxy.google.com/~r/breitbart/~3/1k...,Conway: Kavanaugh Accuser Shouldn't Be 'Insult...,[Ian Hanchett],Breitbart,On Monday’s broadcast of the Fox News Channel’...,False,False,243,"{'senate judiciary committee': 0, 'the whiteho...",3
215,9/17/18,http://feedproxy.google.com/~r/breitbart/~3/K5...,Brett Kavanaugh Ready to Refute 'False Allegat...,[Charlie Spiering],Breitbart,Supreme Court Justice nominee Brett Kavanaugh ...,False,False,227,"{'senate judiciary committee': 2, 'the whiteho...",1
311,9/17/18,http://feedproxy.google.com/~r/breitbart/~3/Z1...,Kellyanne Conway: Brett Kavanaugh Accuser Shou...,[Charlie Spiering],Breitbart,The White House responded Monday to a woman wh...,False,False,212,"{'senate judiciary committee': 1, 'the whiteho...",3
358,10/1/18 6:28,https://www.cnsnews.com/news/article/susan-jon...,Now Democrats Find Fault With Scope of FBI Inv...,[Susan Jones],CNS News,Christine Blasey Ford is greeted by Sen. Mazie...,False,False,1201,"{'senate judiciary committee': 2, 'the whiteho...",7
374,10/5/18 4:30,https://www.cnsnews.com/commentary/patrick-j-b...,We Are All Deplorables Now,[Patrick J. Buchanan],CNS News,President Trump defends his Supreme Court nomi...,False,False,845,"{'senate judiciary committee': 0, 'the whiteho...",1
385,9/24/18 14:44,https://www.cnsnews.com/blog/emily-ward/new-ka...,"New Kavanaugh Accusations: Not One Eyewitness,...",[Emily Ward],CNS News,Supreme Court nominee Brett Kavanaugh. (YouTub...,False,False,721,"{'senate judiciary committee': 1, 'the whiteho...",1
468,9/24/18 17:27,https://www.conservativereview.com/news/cnns-r...,CNNäó»s resident äóÖsleaze balläó» Brian Stelt...,[Rob Eno],conservativereview.com,CNN’s resident ‘sleaze ball’ Brian Stelter act...,False,False,382,"{'senate judiciary committee': 0, 'the whiteho...",1
478,9/27/18 17:26,https://www.conservativereview.com/news/lindse...,Lindsey Graham just made jaws drop at the Kava...,[Chris Pandolfo],conservativereview.com,Lindsey Graham just made jaws drop at the Kava...,False,False,294,"{'senate judiciary committee': 1, 'the whiteho...",1
685,10/5/18 12:14,https://www.dailywire.com/news/36775/kellyanne...,Kellyanne Conway: Longest Confirmation Process...,[],Daily Wire,The fight for Brett Kavanaugh has been the lon...,False,False,260,"{'senate judiciary committee': 0, 'the whiteho...",4
726,9/17/18 12:45,https://www.dailywire.com/news/35942/watch-kel...,WATCH: Kellyanne Conway Says Kavanaugh Accuser...,[],Daily Wire,With Supreme Court nominee Brett Kavanaugh bei...,False,False,280,"{'senate judiciary committee': 2, 'the whiteho...",4


# NER

https://spacy.io/api/annotation#named-entities


In [20]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()


In [23]:
# Example
doc = nlp(df['article'][0])
print([(X.text, X.label_) for X in doc.ents])


[('Democrats', 'NORP'), ('Democrats', 'NORP'), ('Trump', 'ORG'), ('the Carter Page FISA Court', 'ORG'), ('Dianne Feinstein', 'PERSON'), ('the Washington Post', 'ORG'), ('Christine Blasey Ford', 'PERSON'), ('Supreme Court', 'ORG'), ('Brett Kavanaugh', 'PERSON'), ('Georgetown Prep', 'GPE'), ('Mark Judge', 'PERSON'), ('1982', 'DATE'), ('15 years old', 'DATE'), ('Kavanaugh', 'ORG'), ('2012', 'DATE'), ('Kavanaugh', 'PERSON'), ('four', 'CARDINAL'), ('Feinstein', 'PERSON'), ('only two', 'CARDINAL'), ('Blasey Ford', 'PERSON'), ('six years', 'DATE'), ('1982', 'DATE'), ('2012', 'DATE'), ('2012', 'DATE'), ('Washington Post', 'ORG'), ('Kavanaugh', 'PERSON'), ('Kavanaugh', 'GPE'), ('the Supreme Court', 'ORG'), ('Kavanaugh', 'ORG'), ('Supreme Court', 'ORG'), ('Feinstein', 'PERSON'), ('one', 'CARDINAL'), ('four', 'CARDINAL'), ('the year', 'DATE'), ('Patrick J. Smyth', 'PERSON'), ('Christine Blasey Ford', 'PERSON'), ('Washington', 'GPE'), ('Smyth', 'PRODUCT'), ('the Judiciary Committee', 'ORG'), ('tod

In [24]:
def ner_person(article):
    doc = nlp(article)
    os = [(X.text, X.label_) for X in doc.ents]
    results = {}
    for o in os:
        if o[1]=='PERSON':
            if o[0] in results:
                results[o[0]]+=1
            else:
                results[o[0]]=1
    return results

In [25]:
ner_person(df['article'][0])

{'Dianne Feinstein': 1,
 'Christine Blasey Ford': 2,
 'Brett Kavanaugh': 3,
 'Mark Judge': 1,
 'Kavanaugh': 3,
 'Feinstein': 3,
 'Blasey Ford': 1,
 'Patrick J. Smyth': 1,
 'Conrad Black': 1,
 'Andrew C. McCarthy': 1,
 'Kavanagh': 1,
 'McCarthy': 1}

In [26]:
df['people'] = df['article'].apply(lambda x: ner_person(x))

In [27]:
df.to_pickle('resources/data/people_in_articles.pkl')

In [21]:
df.head()

,publish_date,url,title,authors,media_site,article,issue,to_drop,word_count,people
0,9/20/18,https://www.americanthinker.com/articles/2018/...,Blasey Ford Must Be Acknowledged and Then Dism...,[],americanthinker.com,She released only selected portions of her the...,False,False,1266,"{'Dianne Feinstein': 1, 'Christine Blasey Ford..."
1,9/21/18,https://www.americanthinker.com/articles/2018/...,Dear Juanita Broaddrick,[],americanthinker.com,They want an FBI investigation of an individua...,False,False,969,"{'Brett Kavanaugh': 1, 'Christine Blasey Ford'..."
2,9/25/18,https://www.americanthinker.com/articles/2018/...,Debra Katz Was Wrong about Paula Jones's Case,[],americanthinker.com,Debra Katz Was Wrong about Paula Jones's Case\...,False,False,1444,"{'Debra Katz': 2, 'Paula Jones's': 1, 'Christi..."
3,9/28/18,https://www.americanthinker.com/articles/2018/...,Fake Rape Victims Are More Fun than Real Ones,[],americanthinker.com,Fake Rape Victims Are More Fun than Real Ones\...,False,False,1151,"{'Colin Kaepernik': 1, 'Christine Blasey Ford'..."
4,9/20/18,https://www.americanthinker.com/articles/2018/...,Ford vs. Kavanaugh: There's Nothing to Investi...,[],americanthinker.com,The problem with that is that there is nothing...,False,False,934,"{'Christine Blasey Ford's': 3, 'Christine Blas..."


In [28]:
# set of all unique names
holder = {}

for d in list(df['people']):
    for k,v in d.items():
        if k in holder:
            holder[k]+=v
        else:
            holder[k]=v

print(holder)

{'Dianne Feinstein': 1165, 'Christine Blasey Ford': 4222, 'Brett Kavanaugh': 5266, 'Mark Judge': 919, 'Kavanaugh': 12718, 'Feinstein': 1878, 'Blasey Ford': 895, 'Patrick J. Smyth': 66, 'Conrad Black': 2, 'Andrew C. McCarthy': 2, 'Kavanagh': 38, 'McCarthy': 25, 'Bill Clinton': 248, 'Juanita Broaddrick': 92, 'Hillary': 64, 'Broaddrick': 90, 'Christine Ford': 275, 'Rape': 15, 'Clinton': 424, 'Mike Nifong': 2, "Juanita Broaddrick's": 3, 'Daniel John Sobieski': 5, 'Debra Katz': 453, "Paula Jones's": 1, "Christine Blasey Ford's": 211, 'Paula Jones': 51, 'Jones': 73, 'Susan Webber Wright': 1, 'Susan Webber Carter': 1, 'Katz': 312, 'Carter': 15, 'Dave Harrington': 2, 'Dave': 3, 'Harrington': 11, 'Hillary Clinton': 196, 'Steve': 11, 'Allan J. Favish': 1, 'James Fernald': 1, 'Favish': 1, 'Ran': 1, 'Colin Kaepernik': 1, 'Ford': 3892, 'Antonin Scalia': 32, 'MeToo': 335, 'Nikki Yovino': 1, 'Ted Kennedy': 46, 'Tom': 25, 'Conversations': 1, "Bill Clinton's": 8, "Teddy Kennedy's": 1, 'Mary Jo Kopechne

In [32]:
 print(sorted(holder.items(), key = lambda kv:(kv[1], kv[0])))    

[('#Durbin', 1), ('#MeToo', 1), ('- Anita F. Hill', 1), ('- Brett Kavanaugh', 1), ('-Fullerton', 1), ('-Sen', 1), ('1 Tim', 1), ('1/ — John Clune', 1), ('19:5–7', 1), ('2/ —', 1), ('2017 Rosenstein', 1), ('3/ — John Clune', 1), ('44%]. Mucarsel-Powell', 1), ('4Chan', 1), ('@AdamSchindler', 1), ('@Adam_Burtner', 1), ('@Alyssa_Milano', 1), ('@Alyssa_Milano REAL', 1), ('@AmyBrenneman', 1), ('@AnaMariaArchil2', 1), ('@AndrewCFollett', 1), ('@ArthurSchwartz', 1), ('@AstorAaron', 1), ('@AynRandPaulRyan', 1), ('@BenSasse', 1), ('@BiannaGoldryga', 1), ('@BillCosby', 1), ('@BobCorker', 1), ('@BobShank999', 1), ('@BradMossEsq', 1), ('@BretBaier', 1), ('@ChrisChristie', 1), ('@ColludingGoogle', 1), ('@CoreyRobin', 1), ('@CoryBooker', 1), ('@Curtislund2', 1), ('@DanaBashCNN', 1), ('@DavidWright_CNN', 1), ('@DianneFeinstein', 1), ('@DickDurbin', 1), ('@EmilyGorcenski', 1), ('@EricSchultz', 1), ('@GStephanopoulos', 1), ('@GregJaffe', 1), ('@IronStache', 1), ('@JamesArkin', 1), ('@JayCostTWS', 1), ('

In [34]:
x = [('Cindi Nannetti', 11), ('Cristina Miranda King', 11), ('D-W.Va.', 11), ('Darlene Superville', 11), ('DeGeneres', 11), ('Erick Erickson', 11), ('Frank Thorp', 11), ('God', 11), ('Gregg Jarrett', 11), ('Hardwick', 11), ('Harrington', 11), ('Huntsman', 11), ('Jacky Rosen', 11), ('Jeff', 11), ('John Kerry', 11), ('Join', 11), ('Kathy Griffin', 11), ('Lisa Page', 11), ('Louisa Garry', 11), ('Maria Gallagher', 11), ('Martha Kavanaugh', 11), ('Michael', 11), ('Mike Braun', 11), ('Mike Judge', 11), ('Nap', 11), ('P.J.” Smyth', 11), ('Peter Alexander', 11), ('Polygraph', 11), ('Reid', 11), ('Resistance', 11), ('Rob Porter', 11), ('Saul Alinsky', 11), ('Specter', 11), ('Stelter', 11), ('Steve', 11), ('Steve Scalise', 11), ('Sunny Hostin', 11), ('Testify', 11), ('Tina Smith', 11), ('Williams', 11), ('— Sen Dianne Feinstein', 11), ('Adela Gildo-Mazzon', 12), ('Ann Coulter', 12), ('Anthony M. Kennedy', 12), ('Appold', 12), ('Barack Obama’s', 12), ("Blasey Ford's", 12), ('Bruce', 12), ('Charlie Kirk', 12), ('Cuomo', 12), ('Deborah Katz', 12), ('Dino Ewing', 12), ('Driver', 12), ('Eye Injury', 12), ('Gabriel Malor', 12), ('Hyde-Smith', 12), ('Kevin', 12), ('Margaret', 12), ('Martin', 12), ('Matt Rosendale', 12), ('McCaleb', 12), ('Michael R. Bromwich', 12), ('Moon', 12), ('Murray', 12), ('Owens', 12), ('Pence', 12), ('Porter', 12), ('Rachel Maddow', 12), ('Randall', 12), ('Rick Scott', 12), ('Rosa Brooks', 12), ('Rush', 12), ('Ryan', 12), ('Visits', 12), ('Wade', 12), ('Walker', 12), ('Yashar Ali 🐘', 12), ('YouGov', 12), ('— Burgess Everett', 12), ('@DavidRutz\n\n\n\n', 13), ('Alexander', 13), ('Amy Schumer', 13), ('Ari Fleischer', 13), ('Ashley Estes Kavanaugh', 13), ('Brett M. Kavanaugh’s', 13), ('Cheryl Chumley', 13), ('Chris Hayes', 13), ('Chuck Todd', 13), ('Cindy Hyde-Smith', 13), ('Cristina King', 13), ('David French', 13), ('Fleischer', 13), ('Grassley’s', 13), ('Hawley', 13), ('James Roche', 13), ('Jeff Flake’s', 13), ('Joe Biden’s', 13), ('Joel B. Pollak', 13), ("Julie Swetnick's", 13), ('Julie Swetnick’s', 13), ('Kavanugh', 13), ('Keith Koegler', 13), ('LINDSEY GRAHAM', 13), ('Mallory', 13), ('Mary Clare Jalonick', 13), ('Milano', 13), ('Napolitano', 13), ('Sinema', 13), ('Tom Shillue', 13), ('Wray', 13), ('’m', 13), ('Alex Griswold', 14), ('Alito', 14), ('Barbara Boxer', 14), ('Beto O’Rourke', 14), ('Brian Merrick', 14), ('Brian Stelter', 14), ('Butera', 14), ('Carrey', 14), ('Chair Chuck Grassley', 14), ('Dan Murphy', 14), ("Deborah Ramirez's", 14), ('Debra S. Katz', 14), ('Eric Bruce', 14), ('Exit', 14), ('Gallagher', 14), ('Google Adsense', 14), ('Greg', 14), ('Harry Reid', 14), ('Ian Hanchett', 14), ('Jeanine Pirro', 14), ('Joe Scarborough', 14), ('Laura', 14), ('Martha McSally', 14), ('Michael Avenatti’s', 14), ('Patrick “', 14), ('Paul', 14), ('Peter Strzok', 14), ('Robert Costa', 14), ('Robert Mueller’s', 14), ('Ronna McDaniel', 14), ('Ryan Saavedra', 14), ('Sessions', 14), ('Sheila Jackson Lee', 14), ('Tester', 14), ('The Brian Kilmeade Show', 14), ('Twitter @IanHanchett', 14), ('— Guy Benson', 14), ('Bill', 15), ('Bill Clinton’s', 15), ('Bill Montgomery', 15), ('Bryan Robinson', 15), ('Carl', 15), ('Carter', 15), ('Catherine Lucey', 15), ('Enjoy', 15), ('Eric Tucker', 15), ('Fallon', 15), ('Farrakhan', 15), ('Getty Images', 15), ('Google Play', 15), ('Hassan', 15), ('Jim Sciutto', 15), ('Julie', 15), ('Kerry', 15), ('Kim', 15), ('King', 15), ('Maria', 15), ('Maxine Waters', 15), ('Patrick Morrisey', 15), ('Rainford', 15), ('Rape', 15), ('Rosen', 15), ('Rush Limbaugh', 15), ('Thomas’s', 15), ('Thune', 15), ('White', 15), ('Whoopi Goldberg', 15), ('Amy Coney Barrett', 16), ('Banks', 16), ('Brit Hume', 16), ("Christine Ford's", 16), ('D-Vt', 16), ('Dan Bongino', 16), ('Derek Hunter', 16), ('Farrow', 16), ('Garland', 16), ('Henry Rodgers', 16), ('James Hohmann', 16), ('Jesus', 16), ('Kellyanne', 16), ('Ted Lieu', 16), ('Toobin', 16), ('Townhall', 16), ('Charles E. Schumer', 17), ('Gaudette', 17), ('Gorsuch', 17), ('Holton-Arms', 17), ('Jeanine', 17), ('John Nolte', 17), ('John Thune', 17), ('Judiciary Cmte', 17), ('Kav', 17), ('Mike', 17), ('Mike Pompeo', 17), ('Swalwell', 17), ('Swetnik', 17), ('Todd', 17), ('Wilkinson', 17), ('Zeke Miller', 17), ('— Jennifer Rubin', 17), ('Beth Wilkinson', 18), ('Chris Britt', 18), ('Cristina King Miranda', 18), ('Deborah Ramirez’s', 18), ('John', 18), ('Jonathan Turley', 18), ('Juan', 18), ('Kaitlyn Schallhorn', 18), ('Lee', 18), ('Lindsay Graham', 18), ('Loesch', 18), ('Michael Cohen', 18), ('Mike Lee', 18), ('Pam Key', 18), ('Patty Murray', 18), ('Russell', 18), ('Starr', 18), ('Vukmir', 18), ('Wallace', 18), ('@ChuckGrassley', 19), ('@JRubinBlogger', 19), ('Behar', 19), ('Jeryl Bier', 19), ('John McCormack', 19), ('Keith', 19), ('Ketterer', 19), ('Lindsey', 19), ('Maura Kane', 19), ('Paul Manafort', 19), ('Phil Mattingly', 19), ('Roberts', 19), ('Rose', 19), ("Anita Hill's", 20), ('Barbara Van Gelder', 20), ('Below', 20), ('Bill Nelson', 20), ('Bret Baier', 20), ('Carter Page', 20), ('Chris Stirewalt', 20), ('Crenshaw', 20), ('Jeffrey Toobin', 20), ('Jim Acosta', 20), ('Lindsey Graham’s', 20), ('Marsha Blackburn', 20), ('Megyn Kelly', 20), ('O’Rourke', 20), ('Tapper', 20), ('@burgessev', 21), ('@guypbenson', 21), ('Ashley Kavanaugh', 21), ('Ben Sasse', 21), ('Carrie Severino', 21), ('D-Hawaii', 21), ('D-Mass', 21), ('Dolphin', 21), ('Donald J. Trump', 21), ('Emma Brown', 21), ('Garcia', 21), ('George Stephanopoulos', 21), ('INGRAHAM', 21), ('Maura Fitzgerald', 21), ('Michelle Malkin', 21), ('Phil Bredesen', 21), ('Taylor Foy', 21), ('Trump Jr.', 21), ('Webtrends', 21), ('Adam Shaw', 22), ('Bob Menendez', 22), ('Chung', 22), ('David Laufman', 22), ('Fagen', 22), ('George Washington', 22), ('Georgetown Prep', 22), ('James Carville', 22), ('Linda Sarsour', 22), ('Miller', 22), ('Monica L McLean', 22), ('Samuel Chamberlain', 22), ('Sarsour', 22), ('Soros', 22), ('Goldberg', 23), ('Hugh Hewitt', 23), ('Jennifer Rubin', 23), ('John McCain', 23), ('Joy Behar', 23), ('Kim Jong Un', 23), ('Maher', 23), ('Mark Levin', 23), ('Mayer', 23), ('Monica Lee McLean', 23), ('Nadler', 23), ('Patrick', 23), ('Sean Davis', 23), ('Turley', 23), ('Weinstein', 23), ('Brian Fallon', 24), ('Brooke Singman', 24), ('Gillibrand', 24), ('Jackson', 24), ('Matt Damon', 24), ('Matthew Dowd', 24), ('Pradheep J. Shanker', 24), ('Rasor', 24), ('Sotomayor', 24), ('Tim Gaudette', 24), ('Tucker Carlson', 24), ('Woods', 24), ('— Seung Min Kim', 24), ('Anderson Cooper', 25), ('Chuck Grassley’s', 25), ('Dana Loesch', 25), ('ET', 25), ('Joe Arpaio', 25), ('Kerri Kupec', 25), ('McCarthy', 25), ('Nancy Pelosi', 25), ('Nixon', 25), ('Penny Starr', 25), ('Richard Vinneccy', 25), ('Squi', 25), ('Tom', 25), ('Doug Jones', 26), ('Fitzgerald', 26), ('George H.W. Bush', 26), ('Hillary Clinton’s', 26), ('Ingham', 26), ('Jeff Merkley', 26), ('Josh Hawley', 26), ('Ken Starr', 26), ('Monica Lewinsky', 26), ('Roe', 26), ('Scalia', 26), ('Dianne Feinstein’s', 27), ('Kane', 27), ('Leland Ingham Keyser', 27), ('Bernie Sanders', 28), ('Ludington', 28), ('Menendez', 28), ('Paul Sperry', 28), ('Ricki Seidman', 28), ('Rubin', 28), ('— Donald J. Trump', 28), ('Cotton', 29), ('Elena Kagan', 29), ('Levin', 29), ('NICHOLS', 29), ('Robert Mueller', 29), ('Ronald Reagan', 29), ('Tammy Baldwin', 29), ('Whelan', 29), ('Anita Hill’s', 30), ('Ben Shapiro', 30), ('Carlson', 30), ('Cory Gardner', 30), ('Dean Heller', 30), ('Eric Swalwell', 30), ('Fox', 30), ('Kevin Cramer', 30), ('Koegler', 30), ('McSally', 30), ('Al Franken', 31), ('Bloomberg', 31), ('Howard Walsh', 31), ('Jeff Sessions', 31), ('Mike Emanuel', 31), ('Netanyahu', 31), ('Antonin Scalia', 32), ('Christopher Wray', 32), ('D-Minn', 32), ('Laura Ingraham', 32), ('Mitt Romney', 32), ('Andrew Napolitano', 33), ('Blackburn', 33), ('Manafort', 33), ('Steve Daines', 33), ('Blasey-Ford', 34), ('Charles C. W. Cooke', 34), ('Ed', 34), ('Liza', 34), ('Twitchy', 34), ('Alex Pappas', 35), ('Baker', 35), ('McGahn', 35), ('Moore', 35), ('R-Tenn', 35), ('Conway', 36), ('Donald Trump Jr.', 36), ('Heitkamp', 36), ('Kagan', 36), ('Kelly', 36), ('Kimberley Strassel', 36), ('Merkley', 36), ('Ruth Bader Ginsburg', 36), ('Shapiro', 36), ('Twitter', 36), ('Don McGahn', 37), ('KAVANAUGH', 37), ('Bill Cosby', 38), ('Eshoo', 38), ('Kavanagh', 38), ('Sonia Sotomayor', 38), ('Walsh', 38), ('Brown', 39), ('Chris Garrett', 39), ('Charles E. Grassley', 40), ('James Comey', 40), ('Scott', 40), ('Tucker', 40), ('Vinneccy', 40), ('Daines', 41), ('John Clune', 41), ('MacCallum', 41), ('Tom Cotton', 41), ('Clune', 42), ('Ginsburg', 42), ('Jon Tester', 42), ('Kavanuagh', 42), ('Nelson', 42), ('Bork', 43), ('Andrew McCabe', 44), ('D-Calif', 44), ('Harvey Weinstein', 44), ('Mike Davis', 44), ('Smyth', 44), ('John Roberts', 45), ('Reagan', 45), ('Roy Moore', 45), ('Ted Kennedy', 46), ('Chris Wallace', 47), ('Kavanaugh Accuser', 47), ('Coons', 48), ('Russell Ford', 48), ('Seidman', 48), ('Donald Trump’s', 49), ('P.J. Smyth', 49), ('Rescinds Admission Offer', 49), ('Alan Dershowitz', 50), ('Sarah Sanders', 50), ('Hirono', 51), ('Paula Jones', 51), ('Raj Shah', 51), ('Chris', 52), ('Debbie Ramirez', 52), ('Warren', 52), ('Elizabeth Warren', 53), ('Avenatti', 54), ('Baldwin', 54), ('George Soros', 54), ('Gregg Re', 54), ('Mueller', 54), ('Patrick Smyth', 54), ('Christine Ford’s', 55), ('Karen Monahan', 55), ('Charles Grassley', 56), ('Diane Feinstein', 56), ('Mike Pence', 56), ('Stephen Miller', 56), ('Merrick Garland', 57), ('Sarah Huckabee Sanders', 57), ('Claire McCaskill', 58), ('Jane Mayer', 58), ('John Kennedy', 58), ('Donnelly', 59), ('Corker', 60), ('Ingham Keyser', 60), ('Jake Tapper', 61), ('Patrick Leahy', 61), ('Neil Gorsuch', 62), ('GRAHAM', 63), ('Hillary', 64), ('Robert Bork', 64), ('Debbie', 65), ('Klobuchar', 66), ('Patrick J. Smyth', 66), ('Ashley', 67), ('Bredesen', 67), ('Joe Donnelly', 67), ('Blasey Ford’s', 68), ('Trump’s', 68), ('Barack Obama', 69), ('Clintons', 69), ('Leahy', 70), ('Heller', 71), ('Amy Klobuchar', 72), ('McCaskill', 72), ('Ted Cruz', 72), ('Chad Pergram', 73), ('Jones', 73), ('Booker', 74), ('Cornyn', 75), ('Cory Booker', 75), ('Dershowitz', 75), ('Sanders', 75), ('Whitehouse', 77), ('Bromwich', 78), ('George W. Bush', 78), ('Sheldon Whitehouse', 80), ('Joe Biden', 82), ('Stormy Daniels', 82), ('Garrett', 83), ('Anna Eshoo', 84), ('Kirsten Gillibrand', 84), ('Martha MacCallum', 86), ('Dick Durbin', 87), ('R-Ky', 89), ('Ronan Farrow', 89), ('Broaddrick', 90), ('Juanita Broaddrick', 92), ('Monahan', 93), ('Rod Rosenstein', 94), ('Harris', 95), ('Blumenthal', 96), ('Kellyanne Conway', 96), ('Chris Coons', 97), ('McLean', 98), ('Davis', 102), ('Monica McLean', 103), ('Christine', 107), ('Ed Whelan', 108), ('Clarence Thomas', 109), ('Heidi Heitkamp', 109), ('John Cornyn', 109), ('Anthony Kennedy', 111), ('Blasey', 111), ('R-Ariz', 112), ('Durbin', 113), ('Bob Corker', 114), ('Michael Bromwich', 118), ('Cruz', 120), ('Miranda', 120), ('Bush', 121), ('Orrin Hatch', 122), ('Ellison', 125), ('’s', 126), ('Richard Blumenthal', 127), ('Obama', 132), ('Biden', 133), ('Judge', 133), ('Brett M. Kavanaugh', 137), ('Keith Ellison', 137), ('Lisa Banks', 139), ('Hatch', 141), ('Left', 142), ('Kamala Harris', 145), ('Brett Kavanaugh’s', 150), ('Flake', 152), ('Politico', 154), ('Kennedy', 186), ('Hillary Clinton', 196), ('Mark', 199), ('Kavanaugh’s', 201), ('Joe Manchin', 203), ('Schumer', 205), ("Brett Kavanaugh's", 207), ('Rosenstein', 210), ("Christine Blasey Ford's", 211), ('Manchin', 215), ('Chuck Schumer', 217), ('Mazie Hirono', 240), ('Bill Clinton', 248), ('Anita Hill', 249), ('Leland Keyser', 255), ('Facebook', 258), ('Christine Blasey', 271), ('Christine Ford', 275), ('Murkowski', 277), ('Lisa Murkowski', 302), ('Katz', 312), ('Hill', 319), ('MeToo', 335), ('McConnell', 387), ('Mitch McConnell', 389), ('Julie Swetnick', 408), ('Clinton', 424), ('Rachel Mitchell', 433), ('Michael Avenatti', 439), ('Christine Blasey Ford’s', 453), ('Debra Katz', 453), ('Lindsey Graham', 495), ('Collins', 517), ('Susan Collins', 551), ('Brett', 588), ('Thomas', 597), ('Deborah Ramirez', 635), ('Swetnick', 640), ('Jeff Flake', 641), ('Keyser', 670), ('Graham', 719), ('Donald Trump', 746), ('Ramirez', 823), ('Chuck Grassley', 890), ('Blasey Ford', 895), ('Mitchell', 896), ('Mark Judge', 919), ('Trump', 1110), ('Dianne Feinstein', 1165), ('Grassley', 1482), ('Feinstein', 1878), ('Ford', 3892), ('Christine Blasey Ford', 4222), ('Brett Kavanaugh', 5266), ('Kavanaugh', 12718)]

In [35]:
len(x)

613